# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.


In [ ]:
# cleaned data
cleaned_df = df.copy()

# Display data types and first few rows
display(cleaned_df.dtypes)
display(cleaned_df.head())
display(cleaned_df.shape)

# convert measurement timestamp to datetime
cleaned_df["Measurement Timestamp"] = pd.to_datetime(cleaned_df["Measurement Timestamp"])

cleaned_df = cleaned_df.sort_values("Measurement Timestamp").reset_index(drop=True)
display(cleaned_df.dtypes)

#duplicates
duplicates = cleaned_df.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")
#no duplicates

In [ ]:
# handle missing data
missing_data = cleaned_df.isnull().sum()
print("Missing data per column:", missing_data)
cleaned_df["Air Temperature"] = cleaned_df["Air Temperature"].fillna(cleaned_df["Air Temperature"].mean())
cleaned_df["Barometric Pressure"] = cleaned_df["Barometric Pressure"].fillna(cleaned_df["Barometric Pressure"].mean())
display(cleaned_df.isnull().sum())


#drop rows with any remaining missing values
cleaned_df.dropna(inplace=True)
print("Shape after dropping missing values:", cleaned_df.shape)

In [ ]:
# handle outliers
#convert precipitation to categorical
cleaned_df["Precipitation Type"] = cleaned_df["Precipitation Type"].astype('category')
numeric_cols = cleaned_df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    Q1 = cleaned_df[col].quantile(0.25)
    Q3 = cleaned_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = cleaned_df[(cleaned_df[col] < lower_bound) | (cleaned_df[col] > upper_bound)]
    print(f"Number of outliers in {col}: {outliers.shape[0]}")
cols_with_outliers = ["Air Temperature", "Barometric Pressure","Battery Life", "Wind Speed"]
for col in cols_with_outliers:
    Q1 = cleaned_df[col].quantile(0.25)
    Q3 = cleaned_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    print(lower_bound)
    print(upper_bound)
    cleaned_df = cleaned_df[(cleaned_df[col] >= lower_bound) & (cleaned_df[col] <= upper_bound)]
    print(f"Shape after removing outliers in {col}: {cleaned_df.shape}")

#save cleaned data
cleaned_df.to_csv('output/q2_cleaned_data.csv', index=False)
display(cleaned_df.dtypes)

In [ ]:
# report txt file
### 2. `output/q2_cleaning_report.txt`

with open("output/q2_cleaning_report.txt", "w") as f:
    f.write("Data Cleaning Report\n")
    f.write("=================\n")
    f.write(f"Rows before cleaning: {df.shape[0]}\n\n")
    f.write("Missing Data Handled:\n")
    f.write( "- Air Temperature: 75 missing values\n"
        "  Method: filled with mean value\n"
        "  Result: All missing values filled\n\n")
    f.write("-Barometric Pressure: 146 missing values\n" \
        "  Method: filled with mean value\n" \
        "  Result: All missing values filled\n\n")
    f.write("Other missing values: dropped rows with any remaining missing values\n")
    f.write("Outlier Handling:\n")
    f.write("- Air Temperature: Detected 59 outliers using IQR method\n"
        "  Method: Removed at bounds [Q1 - 1.5xIQR, Q3 + 1.5xIQR]\n"
        "  Bounds: [-20.5, 47.5]\n"
        "  Results: 59 values removed\n")
    f.write("- Barometric Pressure: Detected 2796 outliers using IQR method\n"
        "  Method: Removed at bounds [Q1 - 1.5xIQR, Q3 + 1.5xIQR]\n"
        "  Bounds: [978, 1011]\n"
        "  Results: 2796 values removed\n")
    f.write("- Battery Life: Detected 3355 outliers using IQR method\n"
        "  Method: Removed at bounds [Q1 - 1.5xIQR, Q3 + 1.5xIQR]\n"
        "  Bounds: [11.7, 12.1]\n"
        "  Results: 3355 values removed\n")
    f.write("- Wind Speed: Detected 3945 outliers using IQR method\n"
        "  Method: Removed at bounds [Q1 - 1.5xIQR, Q3 + 1.5xIQR]\n"
        "  Bounds: [-2, 7]\n"
        "  Results: 3945 values removed\n\n")
    f.write("Duplicates Removed: 0 (there were no duplicated rows)\n\n")
    f.write("Data Type Coversion:\n"
        "- Measurement Timestamp: converted to datetime\n"
        "  Precipitation Type: converted to categorical\n\n")
    f.write(f"Rows after cleaning: {cleaned_df.shape[0]}")